# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Date List Created
Record num : 32763877
Wall time: 1min 24s


In [3]:
%%time
data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
data['df']['DateTime_frmt']   = pd.to_datetime(data["df"]['DateTime'], format="%Y%m%d %H:%M:%S")
data['df_ohlc'] = data['df'][['DateTime_frmt', 'tick']]

Wall time: 6.73 s


In [4]:
data['df_ohlc'] = data['df'][['DateTime_frmt', 'tick']]
data['candle_size'] = '15T'

In [5]:
%%time
data = get_ohlc(data)

Wall time: 1.65 s


### Hammer

In [6]:
data['df_ohlc']['cdl_hammer'] = talib.CDLHAMMER(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])
data['df_ohlc']['cdl_hammer'] = data['df_ohlc']['cdl_hammer'].replace({100:1})

data['identified_points'] = data['df_ohlc'][data['df_ohlc']['cdl_hammer'] == 1].index
data['test_type'] = 'up'

data = calculate_total_profit(data)

Total Profit        : -6.999999999991296e-05
Num of Transactions : 7
profit Transactions : 3
Loss Transactions   : 4


### Engulfing candle

In [7]:
data['df_ohlc']['cdl_engulfing'] = talib.CDLENGULFING(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])

In [8]:
data['identified_points'] = data['df_ohlc'][data['df_ohlc']['cdl_engulfing'] == 100].index
data['test_type'] = 'up'

data = calculate_total_profit(data)

Total Profit        : -0.1571499999999962
Num of Transactions : 1448
profit Transactions : 588
Loss Transactions   : 860


In [13]:
max(data['df_ohlc']['up_range'][10:15])

0.0004899999999998794

In [9]:
data['identified_points'] = data['df_ohlc'][data['df_ohlc']['cdl_engulfing'] == -100].index
data['test_type'] = 'down'

data = calculate_total_profit(data)

Total Profit        : -0.12165500000001118
Num of Transactions : 1477
profit Transactions : 633
Loss Transactions   : 844


sys.exit()

### Merge with DF to visualize Data

data['df']['cdl_hammer'] = data['df']['cdl_hammer'].replace({0:np.nan})
data['df']['cdl_hammer'] = np.where(data['df']['cdl_hammer'] == 100, data['df']['tick'], data['df']['cdl_hammer'])
data = merge_ohlc_data(data)

### Plot

data["plot"] = False
data['plot_type'] = 'file'
data['reduce_plot'] = True
data['plot_start']  = '2022-07-20'
data['plot_stop']   = '2022-07-21'
data['things_to_plot'] = ['tick', 'positions']

plot_graph(data)